# تبدیل فوریه کوانتومی (QFT)

**Disclaimer**: توی حوزه‌های کوانتومی، دوتا QFT داریم، یکی‌ش همین Quantum Fourier Transformه و اون‌یکی Quantum Field Theory. سعی کنید با هم قاطی‌شون نکنید. :))

In [ ]:
from qiskit import QuantumCircuit, QuantumRegister, ClassicalRegister, assemble, Aer, transpile
from qiskit.visualization import plot_bloch_multivector, plot_histogram, array_to_latex
from qiskit.quantum_info import random_statevector
import numpy as np
import warnings
warnings.filterwarnings('ignore')

sim = Aer.get_backend('aer_simulator')
%matplotlib

In [ ]:
def get_statevector(qc: QuantumCircuit) -> np.ndarray:
    qc.save_statevector()
    qobj = assemble(qc)
    state = sim.run(qobj).result().get_statevector(qc)
    return state


## ۱. مقدمه

همون‌طور که می‌دونیم، تبدیل فوریه توی قسمت‌های مختلفی از علوم کامپیوتر نقش داره. تبدیل فوریه کوانتومی، پیاده‌سازی کوانتومی‌ای از تبدیل فوریه گسسته روی amplitudeهای یه تابع موجه و قسمت مهمی از خیلی الگوریتم‌های کوانتومی‌ه، از جمله الگوریتم فاکتورگیری Shor و الگوریتم Quantum Phase Estimation.

تبدیل فوریه گسسته کلاسیک روی یه بردار  $(x_0, ..., x_{N-1})$ عمل می‌کنه و اون رو طبق فرمول زیر به بردار $(y_0, ..., y_{N-1})$  مپ می‌کنه

$$y_k = \frac{1}{\sqrt{N}}\sum_{j=0}^{N-1}x_j\omega_N^{jk} \space \space ; \space \space \text{where} \space \omega_N^{jk} = e^{2\pi i \frac{jk}{N}} $$

به طور مشابه، می‌شه QFT رو به این صورت تعریف کنیم که روی $\vert X\rangle = \sum_{j=0}^{N-1} x_j \vert j \rangle$ عمل می‌کنه و طبق فرمول زیر اون رو به $\vert Y\rangle = \sum_{k=0}^{N-1} y_k \vert k \rangle$ می‌بره.

$$y_k = \frac{1}{\sqrt{N}}\sum_{j=0}^{N-1}x_j\omega_N^{jk}$$

توجه کنید که فقط amplitude استیت‌هامون تغییر می‌کنن. (می‌تونید این‌طوری بهش فکر کنید که استیت جدیدی ایجاد نمی‌شه یا از بین نمی‌ره، همون استیت‌هایی که از قبل داشتیم رو بعد از تبدیل فوریه‌مون هم داریم، فقط با ضریب‌های متفاوت.)

### تمرین:

توی این تمرین می‌خوایم یه مقدار بیش‌تر و عملی‌تر با QFT آشنا بشیم و حالت‌های مختلف و نحوه ساخت مدار کوانتومی‌ش رو بهتر بشناسیم.

#### ۱. QFT تک‌کیوبیتی
**ورودی:** مدار تک‌کیوبیتی‌ای داریم که توی وضعیت $|\psi\rangle = x_0 |0\rangle + x_1 |1\rangle$ قرار داره.

**هدف:** QFT رو به این کیوبیت اعمال کنید، یعنی ببریدش به این وضعیت:
$$\frac{1}{\sqrt{2}} \big((x_0 + x_1) |0\rangle + (x_0 - x_1) |1\rangle\big)$$

یا اگه یه جور دیگه بخوایم بگیم، پایه‌ی $|j\rangle$ رو ببرید به وضعیت:
 $\frac{1}{\sqrt{2}} \big(|0\rangle + e^{2\pi i \cdot \frac{j}{2}}|1\rangle\big)$

In [ ]:
qc = QuantumCircuit(1)
qc.initialize(random_statevector(2))

def single_QFT(qc: QuantumCircuit):
    pass

#### ۲. گیت دوران یا Rotation
**ورودی‌ها:**

۱. یه مدار تک کیوبیتی توی وضعیت $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$.

۲. یه عدد صحیح $.k \geq 0$

**هدف:**
تابعی بنویسید که مدار رو بگیره و وضعیت کیوبیتش رو به این وضعیت تغییر بدید:
$\alpha |0\rangle + \beta \cdot e^{\frac{2\pi i}{2^{k}}} |1\rangle$
نکته: توی این سوال به طور خاص، نمی‌خوایم global phaseای به مدارمون اضافه شه، پس حواس‌تون بهش باشه.

اگه نمی‌دونید باید چی‌کار کنید، می‌تونید از مستندات کلاس [QuantumCircuit](https://qiskit.org/documentation/stubs/qiskit.circuit.QuantumCircuit.html) استفاده کنید. :دی

In [ ]:
qc = QuantumCircuit(1)
qc.initialize(random_statevector(2))

k = np.random.randint(1, 100)

def qubit_rotation(qc: QuantumCircuit, k: int):
    pass

#### ۳. توان کسر باینری با ورودی کلاسیک.

**ورودی‌ها:**

۱. یه مدار تک کیوبیتی توی وضعیت $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$.

۲. یه آرایه $n$ بیتی از اعداد باینری:

$$[j_1, j_2, ..., j_n] \space \text{where} \space j_k \in \{0,1\} $$

**هدف:**
وضعیت کیوبیت رو به $\alpha |0\rangle + \beta \cdot e^{2\pi i \cdot 0.j_1 j_2 ... j_n} |1\rangle$ تغییر بدید، که توی این فرمول، $0.j_1 j_2 ... j_n$ یه کسر باینریه، مشابه کسرهای ده‌دهی، یعنی:

$$0.j_1 j_2 ... j_n = j_1 \cdot \frac{1}{2^1} + j_2 \cdot \frac{1}{2^2} + ... j_n \cdot \frac{1}{2^n}$$

In [ ]:
qc = QuantumCircuit(1)
qc.initialize(random_statevector(2))

n = 10
j = np.random.randint(0, 2, size=n)

def binary_fraction_classical(qc: QuantumCircuit, j: np.ndarray):
    pass

#### ۴. توان کسر باینری با ورودی کوانتومی.

**ورودی‌ها:**

۱. یه رجیستر تک کیوبیتی توی وضعیت $|\psi\rangle = \alpha |0\rangle + \beta |1\rangle$.

۲. یه رجیستر $n$ تایی از کیوبیت‌ها توی وضعیت $|j_1 j_2 ... j_n\rangle$.

۳. یه مدار کوانتومی که این دو رجیستر توشن.

**هدف:**
وضعیت ورودی رو از 
$(\alpha |0\rangle + \beta |1\rangle) \otimes |j_1 j_2 ... j_n\rangle$
به
$(\alpha |0\rangle + \beta \cdot e^{2\pi i \cdot 0.j_1 j_2 ... j_n} |1\rangle) \otimes |j_1 j_2 ... j_n\rangle$
تغییر بدید، به صورتی که $0.j_1 j_2 ... j_n$ یه کسر باینری به نسبت وضعیت 
$j_1 j_2 ... j_n$ رجیسترمون‌ه.

توجه داشته‌باشید که رجیستر کوانتومیه، یعنی می‌تونه وضعیتش توی Superposition هم باشه، ولی فقط لازمه بتونیم رفتار یه مدار روی پایه‌های برداری‌مون رو کنترل کنیم و بقیه‌ش آسون می‌شه. 

In [75]:
n = 10
psi = QuantumRegister(1, name='psi')
j = QuantumRegister(n, name='j')
qc = QuantumCircuit(psi, j)

j_x = np.random.randint(0, 2, size=n)
j_h = np.random.randint(0, 2, size=n)
for i, (x, h) in enumerate(zip(j_x, j_h)):
    if x:
        qc.x(j[i])
    if h:
        qc.h(j[i])

def binary_fraction_quantum(qc: QuantumCircuit, psi: QuantumRegister, j: QuantumRegister):
    pass

#### ۵. توان کسر باینری با ورودی کوانتومی. (بدون کیوبیت اضافی)

**ورودی‌ها:**

۱. یه رجیستر $n$ تایی از کیوبیت‌ها توی وضعیت $|j_1 j_2 ... j_n\rangle$.

۲. یه مدار کوانتومی که فقط همین رجیستر توش قرار گرفته.

**هدف:**
وضعیت ورودی رو 
$|j_1 j_2 ... j_n\rangle$
به
 $\frac{1}{\sqrt{2}} \big(|0\rangle + e^{2\pi i \cdot 0.j_1 j_2 ... j_n} |1\rangle \otimes |j_2 ... j_n\rangle\big)$
تغییر بدید، به صورتی که $0.j_1 j_2 ... j_n$ یه کسر باینری به نسبت وضعیت 
$j_1 j_2 ... j_n$ رجیسترمون‌ه.

(این تمرین عملا همون تمرین قبلیه، با این فرق که داریم encoding رو توی همون رجیستری که داشتیم انجام می‌دیم.

In [84]:
n = 10
j = QuantumRegister(n, name='j')
qc = QuantumCircuit(j)

j_x = np.random.randint(0, 2, size=n)
j_h = np.random.randint(0, 2, size=n)
for i, (x, h) in enumerate(zip(j_x, j_h)):
    if x:
        qc.x(j[i])
    if h:
        qc.h(j[i])

def binary_fraction_quantum_in_place(qc: QuantumCircuit, j: QuantumRegister):
    pass

#### ۶. برعکس کردن کیوبیت‌ها

**ورودی:**

۱. یه رجیستر $n$ کیوبیتی توی وضعیت $|x_1 x_2 ... x_n \rangle$.

**هدف:**
رجیستر رو ببرید به وضعیت برعکسش، یعنی  $|x_n ... x_2 x_1\rangle$ 

In [ ]:
n = 3
j = QuantumRegister(n, name='j')
qc = QuantumCircuit(j)

j_x = np.random.randint(0, 2, size=n)
for i, x, in enumerate(j_x):
    if x:
        qc.x(j[i])
    else:
        pass
    
def reverse_register(qc: QuantumCircuit, reg: QuantumRegister):
    pass

#### ۷. در نهایت، تبدیل فوریه کوانتومی!

**ورودی:**

۱. یه رجیستر از کیوبیت‌ها توی وضعیت $|j_1 j_2 ... j_n \rangle$.

**هدف:**
اجرای تبدیل فوریه کوانتومی روی این رجیستر، یعنی وضعیت نهایی رجیستر به صورت زیر در بیاد:

$$\frac{1}{\sqrt{2^{n}}} \sum_{k=0}^{n-1} e^{2\pi i \cdot \frac{jk}{2^{n}}} |k\rangle = $$
$$\begin{align}= &\frac{1}{\sqrt{2}} \big(|0\rangle + e^{2\pi i \cdot 0.j_n} |1\rangle\big) \otimes \\
\otimes &\frac{1}{\sqrt{2}} \big(|0\rangle + e^{2\pi i \cdot 0.j_{n-1} j_n} |1\rangle\big) \otimes ... \\
\otimes &\frac{1}{\sqrt{2}} \big(|0\rangle + e^{2\pi i \cdot 0.j_1 j_2 ... j_{n-1} j_n} |1\rangle\big)\end{align}$$

<details>
  <summary><b>برای دیدن راهنمایی این‌جا رو کلیک کنید. </b></summary>
سعی کنید اول یه وضعیت دیگه رو درست کنید و بعد ببریدش به وضعیت نهایی:
    
$\frac{1}{\sqrt{2}} \big(|0\rangle + exp(2\pi i \cdot 0.j_1 j_2 ... j_{n-1} j_n) |1\rangle\big) \otimes ...
\otimes \frac{1}{\sqrt{2}} \big(|0\rangle + exp(2\pi i \cdot 0.j_{n-1} j_n) |1\rangle\big)
\otimes \frac{1}{\sqrt{2}} \big(|0\rangle + exp(2\pi i \cdot 0.j_n) |1\rangle\big)$
    
</details>

In [85]:
n = 10
j = QuantumRegister(n, name='j')
qc = QuantumCircuit(j)

j_x = np.random.randint(0, 2, size=n)
j_h = np.random.randint(0, 2, size=n)
for i, (x, h) in enumerate(zip(j_x, j_h)):
    if x:
        qc.x(j[i])
    if h:
        qc.h(j[i])

def quantum_fourier_transform(qc: QuantumCircuit, j: QuantumRegister):
    pass